In [8]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import make_scorer, mean_squared_error

In [2]:
#JGN DELETE
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

# Load the training and testing data
train_data = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/Bitcoin/Bitcoin Training Testing Call.xlsx", sheet_name="Training Data Call")
test_data = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/Bitcoin/Bitcoin Training Testing Call.xlsx", sheet_name="Testing Data Call")

# Prepare features and target for training
x_train_lgb = train_data[['Asset Price', 'Maturity', 'Strike', 'RVlag1', 'RVlag7', 'RVlag15', 'RVlag30', 
                          'BVlag1', 'BVlag7', 'BVlag15', 'BVlag30', 'SJVlag7', 'SJVlag15', 'SJVlag30']].values
y_train_lgb = train_data['Last'].values

# Prepare features and target for testing
x_test_lgb = test_data[['Asset Price', 'Maturity', 'Strike', 'RVlag1', 'RVlag7', 'RVlag15', 'RVlag30', 
                        'BVlag1', 'BVlag7', 'BVlag15', 'BVlag30', 'SJVlag7', 'SJVlag15', 'SJVlag30']].values
y_test_lgb = test_data['Last'].values

# Define the LightGBM parameters, including early stopping within params
params = {
    'boost_from_average': True,
    'colsample_bytree': 0.5,
    'fair_c': 1,
    'huber_delta': 1,
    'learning_rate': 0.05,
    'max_bin': 255,
    'max_delta_step': 0.7,
    'max_depth': -1,  # None is represented by -1 in LightGBM
    'min_child_samples': 10,
    'min_child_weight': 5,
    'min_split_gain': 0,
    'n_estimators': 2500,
    'num_leaves': 16,
    'objective': 'gamma',
    'reg_alpha': 0,
    'reg_lambda': 0,
    'subsample': 1,
    'subsample_for_bin': 50000,
    'subsample_freq': 1,
    'tweedie_variance_power': 1.5,
    'early_stopping_round': 200  # Set early stopping in parameters
}

# Create LightGBM datasets
train_set = lgb.Dataset(x_train_lgb, label=y_train_lgb)
test_set = lgb.Dataset(x_test_lgb, label=y_test_lgb, reference=train_set)

# Train the model without separate early_stopping_rounds
model = lgb.train(
    params, 
    train_set, 
    valid_sets=[train_set, test_set], 
    valid_names=["train", "test"], 
    num_boost_round=2500
)

# Predict on test data
y_pred_lgb = model.predict(x_test_lgb, num_iteration=model.best_iteration)

# Calculate RMSE for evaluation
rmse = mean_squared_error(y_test_lgb, y_pred_lgb, squared=False)
print(f"RMSE on test set: {rmse}")

/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000537 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3374
[LightGBM] [Info] Number of data points in the train set: 8228, number of used features: 14
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.399805
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.77734	test's gamma: 9.79754
RMSE on test set: 322.09373530290225


In [3]:
y_pred_lgb = model.predict(x_test_lgb, num_iteration=model.best_iteration)

In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Calculate RMSE
rmse = mean_squared_error(y_test_lgb, y_pred_lgb, squared=False)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test_lgb, y_pred_lgb)

# Display RMSE and MAPE
print(f"RMSE on test set: {rmse}")
print(f"MAPE on test set: {mape * 100:.2f}%")


RMSE on test set: 322.09373530290225
MAPE on test set: 2.04%


In [33]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import make_scorer, mean_squared_error

# Define a small grid to tune
param_grid = {
    'learning_rate': [0.05, 0.07],     # Slightly vary the learning rate
    'num_leaves': [16, 24, 32],              # Try a few values around 16
    'max_depth': [-1, 10, 20],               # Include a capped depth as options
    'min_child_samples': [5, 10, 20]         # Slight variations in child samples
}

# Define LightGBM regressor
lgb_model = lgb.LGBMRegressor(
    boosting_type='gbdt',
    objective='gamma',
    boost_from_average=True,
    colsample_bytree=0.5,
    fair_c=1,
    huber_delta=1,
    max_bin=255,
    max_delta_step=0.7,
    n_estimators=2500,
    reg_alpha=0,
    reg_lambda=0,
    subsample=1,
    subsample_for_bin=50000,
    subsample_freq=1,
    tweedie_variance_power=1.5
)

# Define RMSE as the scoring function
scorer = make_scorer(mean_squared_error, squared=False)

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=lgb_model, param_grid=param_grid, cv=5, scoring=scorer, verbose=1)

# Run the grid search
grid_search.fit(x_train_lgb, y_train_lgb)

# Output the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE Score:", grid_search.best_score_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001090 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3373
[LightGBM] [Info] Number of data points in the train set: 6582, number of used features: 14
[LightGBM] [Info] Start training from score 9.400809
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000334 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3368
[LightGBM] [Info] Number of data points in the train set: 6582, number of used features: 14
[LightGBM] [Info] Start training from score 9.403

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000422 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3362
[LightGBM] [Info] Number of data points in the train set: 6582, number of used features: 14
[LightGBM] [Info] Start training from score 9.397688
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000214 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3360
[LightGBM] [Info] Number of data points in the train set: 6583, number of used features: 14
[LightGBM] [Info] Start training from score 9.386857
[Light

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000233 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3368
[LightGBM] [Info] Number of data points in the train set: 6583, number of used features: 14
[LightGBM] [Info] Start training from score 9.410195
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000200 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3373
[LightGBM] [Info] Number of data points in the train set: 6582, number of used features: 14
[LightGBM] [Info] Start training from score 9.400809
[Light

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000196 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3368
[LightGBM] [Info] Number of data points in the train set: 6582, number of used features: 14
[LightGBM] [Info] Start training from score 9.403328
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000253 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3362
[LightGBM] [Info] Number of data points in the train set: 6582, number of used features: 14
[LightGBM] [Info] Start training from score 9.397688
[Light

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000191 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3360
[LightGBM] [Info] Number of data points in the train set: 6583, number of used features: 14
[LightGBM] [Info] Start training from score 9.386857
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000389 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3368
[LightGBM] [Info] Number of data points in the train set: 6583, number of us

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000200 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3368
[LightGBM] [Info] Number of data points in the train set: 6582, number of used features: 14
[LightGBM] [Info] Start trainin

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000190 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3360
[LightGBM] [Info] Number of data points in the train set: 6583, number of used features: 14
[LightGBM] [Info] Start training from score 9.386857
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000194 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3373
[LightGBM] [Info] Number of data points in the train set: 6582, number of used features: 14
[LightGBM] [Info] Start training from score 9.400809
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000192 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3368
[LightGBM] [Info] Number of data points in the train set: 6582, number of used features: 14
[LightGBM] [Info] Start training from score 9.403328
[Light

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000193 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3362
[LightGBM] [Info] Number of data points in the train set: 6582, number of used features: 14
[LightGBM] [Info] Start training from score 9.397688
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000230 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3360
[LightGBM] [Info] Number of data points in the train set: 6583, number of used features: 14
[LightGBM] [Info] Start training from score 9.386857
[Light

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000230 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3368
[LightGBM] [Info] Number of data points in the train set: 6583, number of used features: 14
[LightGBM] [Info] Start training from score 9.410195
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000254 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3374
[LightGBM] [Info] Number of data points in the train set: 8228, number of used features: 14
[LightGBM] [Info] Start training from score 9.399805
Best P

In [12]:
# Step 2: Predict on the test set using the best parameters
best_params = grid_search.best_params_

# Initialize the model with the best parameters
best_model = lgb.LGBMRegressor(**best_params,
                                 boosting_type='gbdt',
                                 objective='gamma',
                                 boost_from_average=True,
                                 colsample_bytree=0.5,
                                 fair_c=1,
                                 huber_delta=1,
                                 max_bin=255,
                                 max_delta_step=0.7,
                                 n_estimators=2500,
                                 reg_alpha=0,
                                 reg_lambda=0,
                                 subsample=1,
                                 subsample_for_bin=50000,
                                 subsample_freq=1,
                                 tweedie_variance_power=1.5)

# Fit the model to the training data
best_model.fit(x_train_lgb, y_train_lgb)

# Predict on the test data
y_pred_lgb = best_model.predict(x_test_lgb)

# Calculate RMSE for evaluation
rmse = mean_squared_error(y_test_lgb, y_pred_lgb, squared=False)
print(f"RMSE on test set: {rmse}")

# Optional: Calculate MAPE
mape = np.mean(np.abs((y_test_lgb - y_pred_lgb) / y_test_lgb)) * 100
print(f"MAPE on test set: {mape}%")

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001578 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3374
[LightGBM] [Info] Number of data points in the train set: 8228, number of used features: 14
[LightGBM] [Info] Start training from score 9.399805
[LightGBM] [Warning] Unknown parameter: huber_delta
RMSE on test set: 313.57618977018376
MAPE on test set: 2.030915058693726%


In [15]:
dates = test_data['Time']  # Replace 'Date' with the correct column name for the dates

# Prepare the results DataFrame
results = pd.DataFrame({
    'Date': dates,                 # Dates from the test dataset
    'Actual': y_test_lgb,          # Actual values
    'Predicted': y_pred_lgb        # Predicted values
})

# Save the results to an Excel file
output_file = "/Users/shannenwibisono/Desktop/-Skripsi-/Results LGBM BTC DR and GridSCV Call.xlsx"
results.to_excel(output_file, index=False)

print(f"Results saved to Excel file: {output_file}")

Results saved to Excel file: /Users/shannenwibisono/Desktop/-Skripsi-/Results LGBM BTC DR and GridSCV Call.xlsx


In [ ]:
#PUT OPTION#

In [16]:
# Read training and testing data for Put options
train_data_put = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/Bitcoin/Bitcoin Training Testing Put.xlsx", sheet_name="Training Data Put")
test_data_put = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/Bitcoin/Bitcoin Training Testing Put.xlsx", sheet_name="Testing Data Put")

# Prepare features and target for training (Put)
x_train_lgb_put = train_data_put[['Asset Price', 'Maturity', 'Strike', 'RVlag1', 'RVlag7', 'RVlag15', 'RVlag30', 
                                  'BVlag1', 'BVlag7', 'BVlag15', 'BVlag30', 'SJVlag7', 'SJVlag15', 'SJVlag30']].values
y_train_lgb_put = train_data_put['Last'].values

# Prepare features and target for testing (Put)
x_test_lgb_put = test_data_put[['Asset Price', 'Maturity', 'Strike', 'RVlag1', 'RVlag7', 'RVlag15', 'RVlag30', 
                                'BVlag1', 'BVlag7', 'BVlag15', 'BVlag30', 'SJVlag7', 'SJVlag15', 'SJVlag30']].values
y_test_lgb_put = test_data_put['Last'].values

In [17]:
# Define the LightGBM parameters
params = {
    'boost_from_average': True,
    'colsample_bytree': 0.5,
    'fair_c': 1,
    'huber_delta': 1,
    'learning_rate': 0.05,
    'max_bin': 255,
    'max_delta_step': 0.7,
    'max_depth': -1,  # None is represented by -1 in LightGBM
    'min_child_samples': 10,
    'min_child_weight': 5,
    'min_split_gain': 0,
    'n_estimators': 2500,
    'num_leaves': 16,
    'objective': 'gamma',
    'reg_alpha': 0,
    'reg_lambda': 0,
    'subsample': 1,
    'subsample_for_bin': 50000,
    'subsample_freq': 1,
    'tweedie_variance_power': 1.5,
    'early_stopping_round': 200  # Set early stopping in parameters
}

# Create LightGBM datasets
train_set = lgb.Dataset(x_train_lgb_put, label=y_train_lgb_put)
test_set = lgb.Dataset(x_test_lgb_put, label=y_test_lgb_put, reference=train_set)

# Train the model without separate early_stopping_rounds
model = lgb.train(
    params, 
    train_set, 
    valid_sets=[train_set, test_set], 
    valid_names=["train", "test"], 
    num_boost_round=2500
)

# Predict on test data
y_pred_lgb_put = model.predict(x_test_lgb_put, num_iteration=model.best_iteration)

# Calculate RMSE for evaluation
rmse = mean_squared_error(y_test_lgb_put, y_pred_lgb_put, squared=False)
print(f"RMSE on test set: {rmse}")

# Calculate MAPE
mape = np.mean(np.abs((y_test_lgb_put - y_pred_lgb_put) / y_test_lgb_put)) * 100
print(f"MAPE on test set: {mape:.2f}%")

/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002122 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3376
[LightGBM] [Info] Number of data points in the train set: 8446, number of used features: 14
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.242772
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.08725	test's gamma: 8.97394
RMSE on test set: 343.2434748788248
MAPE on test set: 3.79%


In [18]:
dates = test_data_put['Time']  # Replace 'Date' with the correct column name for the dates

# Prepare the results DataFrame
results = pd.DataFrame({
    'Date': dates,                 # Dates from the test dataset
    'Actual': y_test_lgb_put,          # Actual values
    'Predicted': y_pred_lgb_put        # Predicted values
})

# Save the results to an Excel file
output_file = "/Users/shannenwibisono/Desktop/-Skripsi-/Results LGBM BTC DataRobot Put.xlsx"
results.to_excel(output_file, index=False)

print(f"Results saved to Excel file: {output_file}")

Results saved to Excel file: /Users/shannenwibisono/Desktop/-Skripsi-/Results LGBM BTC DataRobot Put.xlsx


In [40]:
# Define a small grid to tune
param_grid = {
    'learning_rate': [0.05, 0.07],     # Slightly vary the learning rate
    'num_leaves': [16, 24, 32],              # Try a few values around 16
    'max_depth': [-1, 10, 20],               # Include a capped depth as options
    'min_child_samples': [5, 10, 20]         # Slight variations in child samples
}

# Define LightGBM regressor
lgb_model = lgb.LGBMRegressor(
    boosting_type='gbdt',
    objective='gamma',
    boost_from_average=True,
    colsample_bytree=0.5,
    fair_c=1,
    huber_delta=1,
    max_bin=255,
    max_delta_step=0.7,
    n_estimators=2500,
    reg_alpha=0,
    reg_lambda=0,
    subsample=1,
    subsample_for_bin=50000,
    subsample_freq=1,
    tweedie_variance_power=1.5
)

# Define RMSE as the scoring function
scorer = make_scorer(mean_squared_error, squared=False)

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=lgb_model, param_grid=param_grid, cv=5, scoring=scorer, verbose=1)

# Run the grid search
grid_search.fit(x_train_lgb_put, y_train_lgb_put)

# Output the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE Score:", grid_search.best_score_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001459 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3370
[LightGBM] [Info] Number of data points in the train set: 6756, number of used features: 14
[LightGBM] [Info] Start training from score 9.249780
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000584 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3364
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features: 14
[LightGBM] [Info] Start training from score 9.228

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000071 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3366
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features: 14
[LightGBM] [Info] Start training from score 9.246527
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000236 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3358
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000244 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3362
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features: 14
[LightGBM] [Info] Start training from score 9.238465
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000254 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3370
[LightGBM] [Info] Number of data points in the train set: 6756, number of used features: 14
[LightGBM] [Info] Start training from score 9.249780
[Light

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000203 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3364
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features: 14
[LightGBM] [Info] Start training from score 9.228815
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000203 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3366
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features: 14
[LightGBM] [Info] Start training from score 9.246527
[Light

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000195 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3358
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features: 14
[LightGBM] [Info] Start training from score 9.250108
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000259 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3362
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features: 14
[LightGBM] [Info] Start training from score 9.238465
[Light

...

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000235 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3370
[LightGBM] [Info] Number of data points in the train set: 6756, number of used features: 14
[LightGBM] [Info] Start training from score 9.249780
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000196 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3364
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features: 14
[LightGBM] [Info] Start training from score 9.228815
[Light

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000197 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3358
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features: 14
[LightGBM] [Info] Start trainin

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000234 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3366
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features: 14
[LightGBM] [Info] Start training from score 9.246527
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000204 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3358
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features: 14
[LightGBM] [Info] Start training from score 9.250108
[Light

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000242 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3362
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features: 14
[LightGBM] [Info] Start training from score 9.238465
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000255 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3370
[LightGBM] [Info] Number of data points in the train set: 6756, number of used features: 14
[LightGBM] [Info] Start training from score 9.249780
[Light

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000210 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3364
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features: 14
[LightGBM] [Info] Start training from score 9.228815
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000197 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3366
[LightGBM] [Info] Number of data points in the train set: 6757, number of used features: 14
[LightGBM] [Info] Start training from score 9.246527
[Light

In [22]:
# Step 2: Predict on the test set using the best parameters
best_params = grid_search.best_params_

# Initialize the model with the best parameters
best_model = lgb.LGBMRegressor(**best_params,
                                 boosting_type='gbdt',
                                 objective='gamma',
                                 boost_from_average=True,
                                 colsample_bytree=0.5,
                                 fair_c=1,
                                 huber_delta=1,
                                 max_bin=255,
                                 max_delta_step=0.7,
                                 n_estimators=2500,
                                 reg_alpha=0,
                                 reg_lambda=0,
                                 subsample=1,
                                 subsample_for_bin=50000,
                                 subsample_freq=1,
                                 tweedie_variance_power=1.5)

# Fit the model to the training data
best_model.fit(x_train_lgb_put, y_train_lgb_put)

# Predict on the test data
y_pred_lgb_put = best_model.predict(x_test_lgb_put)

# Calculate RMSE for evaluation
rmse = mean_squared_error(y_test_lgb_put, y_pred_lgb_put, squared=False)
print(f"RMSE on test set: {rmse}")

# Optional: Calculate MAPE
mape = np.mean(np.abs((y_test_lgb_put - y_pred_lgb_put) / y_test_lgb_put)) * 100
print(f"MAPE on test set: {mape:.2f}%")

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000860 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3376
[LightGBM] [Info] Number of data points in the train set: 8446, number of used features: 14
[LightGBM] [Info] Start training from score 9.242772
[LightGBM] [Warning] Unknown parameter: huber_delta
RMSE on test set: 352.8885192277529
MAPE on test set: 3.87%


In [23]:
dates = test_data_put['Time']  # Replace 'Date' with the correct column name for the dates

# Prepare the results DataFrame
results = pd.DataFrame({
    'Date': dates,                 # Dates from the test dataset
    'Actual': y_test_lgb_put,          # Actual values
    'Predicted': y_pred_lgb_put        # Predicted values
})

# Save the results to an Excel file
output_file = "/Users/shannenwibisono/Desktop/-Skripsi-/Results LGBM BTC GridSCV Put.xlsx"
results.to_excel(output_file, index=False)

print(f"Results saved to Excel file: {output_file}")

Results saved to Excel file: /Users/shannenwibisono/Desktop/-Skripsi-/Results LGBM BTC GridSCV Put.xlsx
